<h1 color=#009d71> EDA </h1>

In [1]:
import json
import os
import re
import pandas as pd
import matplotlib.pyplot as plt



In [2]:
with open("../Data/MLA_100k.jsonlines") as jsonfile:
    line = jsonfile.readline()
    print(line)

{"seller_address": {"comment": "", "longitude": -58.3986709, "id": 78162842, "country": {"name": "Argentina", "id": "AR"}, "address_line": "", "latitude": -34.6280698, "search_location": {"neighborhood": {"name": "San Crist\u00f3bal", "id": "TUxBQlNBTjkwNTZa"}, "state": {"name": "Capital Federal", "id": "TUxBUENBUGw3M2E1"}, "city": {"name": "Capital Federal", "id": "TUxBQ0NBUGZlZG1sYQ"}}, "zip_code": "", "city": {"name": "San Crist\u00f3bal", "id": "TUxBQlNBTjkwNTZa"}, "state": {"name": "Capital Federal", "id": "AR-C"}}, "warranty": null, "sub_status": [], "condition": "new", "seller_contact": null, "deal_ids": [], "base_price": 80, "shipping": {"local_pick_up": true, "methods": [], "tags": [], "free_shipping": false, "mode": "not_specified", "dimensions": null}, "non_mercado_pago_payment_methods": [{"description": "Transferencia bancaria", "id": "MLATB", "type": "G"}, {"description": "Acordar con el comprador", "id": "MLAWC", "type": "G"}, {"description": "Efectivo", "id": "MLAMO", "t

In [3]:
df = pd.read_json("../Data/MLA_100k.jsonlines", lines=True, encoding='utf-8')


In [4]:
df.head()

,seller_address,warranty,sub_status,condition,seller_contact,deal_ids,base_price,shipping,non_mercado_pago_payment_methods,seller_id,...,status,video_id,catalog_product_id,subtitle,initial_quantity,start_time,permalink,geolocation,sold_quantity,available_quantity
0,"{'comment': '', 'longitude': -58.3986709, 'id'...",None,[],new,None,[],80.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",74952096,...,active,None,NaN,NaN,1,2015-09-05 20:42:53+00:00,http://articulo.mercadolibre.com.ar/MLA-578052...,"{'latitude': -34.6280698, 'longitude': -58.398...",0,1
1,"{'comment': '', 'longitude': -58.5059173, 'id'...",NUESTRA REPUTACION,[],used,None,[],2650.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",42093335,...,active,None,NaN,NaN,1,2015-09-26 18:08:30+00:00,http://articulo.mercadolibre.com.ar/MLA-581565...,"{'latitude': -34.5935524, 'longitude': -58.505...",0,1
2,"{'comment': '', 'longitude': -58.4143948, 'id'...",None,[],used,None,[],60.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",133384258,...,active,None,NaN,NaN,1,2015-09-09 23:57:07+00:00,http://articulo.mercadolibre.com.ar/MLA-578780...,"{'latitude': -34.6233907, 'longitude': -58.414...",0,1
3,"{'comment': '', 'longitude': -58.4929208, 'id'...",None,[],new,None,[],580.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",143001605,...,active,None,NaN,NaN,1,2015-09-28 18:47:56+00:00,http://articulo.mercadolibre.com.ar/MLA-581877...,"{'latitude': -34.6281894, 'longitude': -58.492...",0,1
4,"{'comment': '', 'longitude': -58.5495042, 'id'...",MI REPUTACION.,[],used,None,[],30.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",96873449,...,active,None,NaN,NaN,1,2015-08-24 22:07:20+00:00,http://articulo.mercadolibre.com.ar/MLA-576112...,"{'latitude': -34.6346547, 'longitude': -58.549...",0,1


In [5]:
df.shape

(100000, 48)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 48 columns):
 #   Column                            Non-Null Count   Dtype              
---  ------                            --------------   -----              
 0   seller_address                    100000 non-null  object             
 1   warranty                          39104 non-null   object             
 2   sub_status                        100000 non-null  object             
 3   condition                         100000 non-null  object             
 4   seller_contact                    2219 non-null    object             
 5   deal_ids                          100000 non-null  object             
 6   base_price                        100000 non-null  float64            
 7   shipping                          100000 non-null  object             
 8   non_mercado_pago_payment_methods  100000 non-null  object             
 9   seller_id                         100000 non-null

In [48]:
df.describe()

,base_price,seller_id,price,official_store_id,differential_pricing,original_price,catalog_product_id,subtitle,initial_quantity,sold_quantity,available_quantity
count,1.000000e+05,1.000000e+05,1.000000e+05,818.000000,0.0,143.000000,1.100000e+01,0.0,100000.000000,100000.000000,100000.000000
mean,5.252423e+04,8.425269e+07,5.252433e+04,206.443765,NaN,1593.341958,3.727643e+06,NaN,35.093370,2.396990,34.842380
std,8.623127e+06,5.497257e+07,8.623127e+06,128.252953,NaN,2245.798072,1.884698e+06,NaN,421.076196,42.685077,420.808403
min,8.400000e-01,1.304000e+03,8.400000e-01,1.000000,NaN,120.000000,9.440400e+04,NaN,1.000000,0.000000,1.000000
25%,9.000000e+01,3.953591e+07,9.000000e+01,84.000000,NaN,449.000000,3.050702e+06,NaN,1.000000,0.000000,1.000000
50%,2.500000e+02,7.631063e+07,2.500000e+02,216.000000,NaN,858.000000,5.093232e+06,NaN,1.000000,0.000000,1.000000
75%,8.000000e+02,1.325659e+08,8.000000e+02,312.750000,NaN,1500.000000,5.103216e+06,NaN,2.000000,0.000000,2.000000
max,2.222222e+09,1.946906e+08,2.222222e+09,446.000000,NaN,13999.000000,5.434513e+06,NaN,9999.000000,8676.000000,9999.000000


In [7]:
df.keys()

Index(['seller_address', 'warranty', 'sub_status', 'condition',
       'seller_contact', 'deal_ids', 'base_price', 'shipping',
       'non_mercado_pago_payment_methods', 'seller_id', 'variations',
       'location', 'site_id', 'listing_type_id', 'price', 'attributes',
       'buying_mode', 'tags', 'listing_source', 'parent_item_id',
       'coverage_areas', 'category_id', 'descriptions', 'last_updated',
       'international_delivery_mode', 'pictures', 'id', 'official_store_id',
       'differential_pricing', 'accepts_mercadopago', 'original_price',
       'currency_id', 'thumbnail', 'title', 'automatic_relist', 'date_created',
       'secure_thumbnail', 'stop_time', 'status', 'video_id',
       'catalog_product_id', 'subtitle', 'initial_quantity', 'start_time',
       'permalink', 'geolocation', 'sold_quantity', 'available_quantity'],
      dtype='object')

In [8]:
dfkeys = list(df.keys())
index_keys =  {f"{i}":dfkeys[i] for i in range(len(dfkeys))}


In [9]:
print(index_keys)

{'0': 'seller_address', '1': 'warranty', '2': 'sub_status', '3': 'condition', '4': 'seller_contact', '5': 'deal_ids', '6': 'base_price', '7': 'shipping', '8': 'non_mercado_pago_payment_methods', '9': 'seller_id', '10': 'variations', '11': 'location', '12': 'site_id', '13': 'listing_type_id', '14': 'price', '15': 'attributes', '16': 'buying_mode', '17': 'tags', '18': 'listing_source', '19': 'parent_item_id', '20': 'coverage_areas', '21': 'category_id', '22': 'descriptions', '23': 'last_updated', '24': 'international_delivery_mode', '25': 'pictures', '26': 'id', '27': 'official_store_id', '28': 'differential_pricing', '29': 'accepts_mercadopago', '30': 'original_price', '31': 'currency_id', '32': 'thumbnail', '33': 'title', '34': 'automatic_relist', '35': 'date_created', '36': 'secure_thumbnail', '37': 'stop_time', '38': 'status', '39': 'video_id', '40': 'catalog_product_id', '41': 'subtitle', '42': 'initial_quantity', '43': 'start_time', '44': 'permalink', '45': 'geolocation', '46': 'so

In [10]:
for i in index_keys:
    print(i, index_keys[i])

0 seller_address
1 warranty
2 sub_status
3 condition
4 seller_contact
5 deal_ids
6 base_price
7 shipping
8 non_mercado_pago_payment_methods
9 seller_id
10 variations
11 location
12 site_id
13 listing_type_id
14 price
15 attributes
16 buying_mode
17 tags
18 listing_source
19 parent_item_id
20 coverage_areas
21 category_id
22 descriptions
23 last_updated
24 international_delivery_mode
25 pictures
26 id
27 official_store_id
28 differential_pricing
29 accepts_mercadopago
30 original_price
31 currency_id
32 thumbnail
33 title
34 automatic_relist
35 date_created
36 secure_thumbnail
37 stop_time
38 status
39 video_id
40 catalog_product_id
41 subtitle
42 initial_quantity
43 start_time
44 permalink
45 geolocation
46 sold_quantity
47 available_quantity


In [11]:
df["status"].unique()

array(['active', 'paused', 'closed', 'not_yet_active'], dtype=object)

In [12]:
df["non_mercado_pago_payment_methods"]

0        [{'description': 'Transferencia bancaria', 'id...
1        [{'description': 'Transferencia bancaria', 'id...
2        [{'description': 'Transferencia bancaria', 'id...
3        [{'description': 'Transferencia bancaria', 'id...
4        [{'description': 'Transferencia bancaria', 'id...
                               ...                        
99995    [{'description': 'Transferencia bancaria', 'id...
99996    [{'description': 'Tarjeta de crédito', 'id': '...
99997                                                   []
99998    [{'description': 'Transferencia bancaria', 'id...
99999    [{'description': 'Transferencia bancaria', 'id...
Name: non_mercado_pago_payment_methods, Length: 100000, dtype: object

In [13]:
df.iloc[0,0]

{'comment': '',
 'longitude': -58.3986709,
 'id': 78162842,
 'country': {'name': 'Argentina', 'id': 'AR'},
 'address_line': '',
 'latitude': -34.6280698,
 'search_location': {'neighborhood': {'name': 'San Cristóbal',
   'id': 'TUxBQlNBTjkwNTZa'},
  'state': {'name': 'Capital Federal', 'id': 'TUxBUENBUGw3M2E1'},
  'city': {'name': 'Capital Federal', 'id': 'TUxBQ0NBUGZlZG1sYQ'}},
 'zip_code': '',
 'city': {'name': 'San Cristóbal', 'id': 'TUxBQlNBTjkwNTZa'},
 'state': {'name': 'Capital Federal', 'id': 'AR-C'}}

In [14]:
df.iloc[0,7]

{'local_pick_up': True,
 'methods': [],
 'tags': [],
 'free_shipping': False,
 'mode': 'not_specified',
 'dimensions': None}

In [15]:
df.iloc[0,8]

[{'description': 'Transferencia bancaria', 'id': 'MLATB', 'type': 'G'},
 {'description': 'Acordar con el comprador', 'id': 'MLAWC', 'type': 'G'},
 {'description': 'Efectivo', 'id': 'MLAMO', 'type': 'G'}]

In [16]:
df["condition"].unique()

array(['new', 'used'], dtype=object)

In [17]:
for i in df:
    print(i)

seller_address
warranty
sub_status
condition
seller_contact
deal_ids
base_price
shipping
non_mercado_pago_payment_methods
seller_id
variations
location
site_id
listing_type_id
price
attributes
buying_mode
tags
listing_source
parent_item_id
coverage_areas
category_id
descriptions
last_updated
international_delivery_mode
pictures
id
official_store_id
differential_pricing
accepts_mercadopago
original_price
currency_id
thumbnail
title
automatic_relist
date_created
secure_thumbnail
stop_time
status
video_id
catalog_product_id
subtitle
initial_quantity
start_time
permalink
geolocation
sold_quantity
available_quantity


In [18]:
json_pattern = r"^\{.*\}$"
tst = re.match(json_pattern, "{hola}")
print(tst)

<re.Match object; span=(0, 6), match='{hola}'>


In [19]:
string_objects = {}
for i in range(48):
    if re.match(json_pattern, str(df.iloc[0,i])):
            string_objects[f"{dfkeys[i]}"] = i

In [20]:
string_objects

{'seller_address': 0, 'shipping': 7, 'location': 11, 'geolocation': 45}

In [21]:
json_pattern2 = r".*\{.*\}*"
string_objects2 = {}
for i in range(48):
    if re.match(json_pattern2, str(df.iloc[0,i])):
        print(df.iloc[0,i])
        string_objects2[f"{dfkeys[i]}"] = i

{'comment': '', 'longitude': -58.3986709, 'id': 78162842, 'country': {'name': 'Argentina', 'id': 'AR'}, 'address_line': '', 'latitude': -34.6280698, 'search_location': {'neighborhood': {'name': 'San Cristóbal', 'id': 'TUxBQlNBTjkwNTZa'}, 'state': {'name': 'Capital Federal', 'id': 'TUxBUENBUGw3M2E1'}, 'city': {'name': 'Capital Federal', 'id': 'TUxBQ0NBUGZlZG1sYQ'}}, 'zip_code': '', 'city': {'name': 'San Cristóbal', 'id': 'TUxBQlNBTjkwNTZa'}, 'state': {'name': 'Capital Federal', 'id': 'AR-C'}}
{'local_pick_up': True, 'methods': [], 'tags': [], 'free_shipping': False, 'mode': 'not_specified', 'dimensions': None}
[{'description': 'Transferencia bancaria', 'id': 'MLATB', 'type': 'G'}, {'description': 'Acordar con el comprador', 'id': 'MLAWC', 'type': 'G'}, {'description': 'Efectivo', 'id': 'MLAMO', 'type': 'G'}]
{}
[{'id': 'MLA578052519-912855983'}]
[{'size': '500x375', 'secure_url': 'https://a248.e.akamai.net/mla-s1-p.mlstatic.com/5386-MLA4352067878_052013-O.jpg', 'max_size': '1200x900', '

In [22]:
print(string_objects2)

{'seller_address': 0, 'shipping': 7, 'non_mercado_pago_payment_methods': 8, 'location': 11, 'descriptions': 22, 'pictures': 25, 'geolocation': 45}


In [23]:
df.iloc[0,22]

[{'id': 'MLA578052519-912855983'}]

In [24]:
df.iloc[0,25]

[{'size': '500x375',
  'secure_url': 'https://a248.e.akamai.net/mla-s1-p.mlstatic.com/5386-MLA4352067878_052013-O.jpg',
  'max_size': '1200x900',
  'url': 'http://mla-s1-p.mlstatic.com/5386-MLA4352067878_052013-O.jpg',
  'quality': '',
  'id': '5386-MLA4352067878_052013'},
 {'size': '500x375',
  'secure_url': 'https://a248.e.akamai.net/mla-s1-p.mlstatic.com/5361-MLA4352068035_052013-O.jpg',
  'max_size': '1200x900',
  'url': 'http://mla-s1-p.mlstatic.com/5361-MLA4352068035_052013-O.jpg',
  'quality': '',
  'id': '5361-MLA4352068035_052013'}]

In [25]:
df.iloc[0,11]

{}

In [26]:
df.iloc[0,45]

{'latitude': -34.6280698, 'longitude': -58.3986709}

In [27]:
df.iloc[0,0]

{'comment': '',
 'longitude': -58.3986709,
 'id': 78162842,
 'country': {'name': 'Argentina', 'id': 'AR'},
 'address_line': '',
 'latitude': -34.6280698,
 'search_location': {'neighborhood': {'name': 'San Cristóbal',
   'id': 'TUxBQlNBTjkwNTZa'},
  'state': {'name': 'Capital Federal', 'id': 'TUxBUENBUGw3M2E1'},
  'city': {'name': 'Capital Federal', 'id': 'TUxBQ0NBUGZlZG1sYQ'}},
 'zip_code': '',
 'city': {'name': 'San Cristóbal', 'id': 'TUxBQlNBTjkwNTZa'},
 'state': {'name': 'Capital Federal', 'id': 'AR-C'}}

In [28]:
def  compare_json_geo(row: pd.DataFrame) -> bool:
    json1 = row["seller_address"]
    json2 = row["geolocation"]
    return (
        json1["longitude"] == json2["longitude"]
    )and(
        json1["latitude"] == json2["latitude"]
    )

In [29]:
df["same_info_0_45"] = df.apply(compare_json_geo, axis=1)

In [30]:
df["same_info_0_45"].value_counts()

same_info_0_45
True     98249
False     1751
Name: count, dtype: int64

In [31]:
differents_locations = df[df["same_info_0_45"] == False][[index_keys["0"],index_keys["45"]]]

In [32]:
differents_locations.iloc[0,0]

{'comment': '',
 'longitude': -58.3716129,
 'id': 154081243,
 'country': {'name': 'Argentina', 'id': 'AR'},
 'address_line': 'Defensa 850',
 'latitude': -34.6177886,
 'search_location': {'neighborhood': {'name': 'San Telmo',
   'id': 'TUxBQlNBTjgxMzNa'},
  'state': {'name': 'Capital Federal', 'id': 'TUxBUENBUGw3M2E1'},
  'city': {'name': 'Capital Federal', 'id': 'TUxBQ0NBUGZlZG1sYQ'}},
 'zip_code': '1065',
 'city': {'name': 'San Telmo', 'id': 'TUxBQlNBTjgxMzNa'},
 'state': {'name': 'Capital Federal', 'id': 'AR-C'}}

In [33]:
differents_locations.iloc[0,1]

{'latitude': '', 'longitude': ''}

In [34]:
x = 0
listfalse = []
listtrue = []
for i in list(differents_locations.index):
    if (differents_locations.loc[i]["seller_address"]["longitude"] == "" or 
        differents_locations.loc[i]["geolocation"]["longitude"] == "") or (
        differents_locations.loc[i]["seller_address"]["latitude"] == "" or 
        differents_locations.loc[i]["geolocation"]["latitude"] == ""):
            listfalse.append(i)
            x+=1
            continue
    listtrue.append(i)
print(x)

1491


In [35]:
print(listfalse)

[59, 261, 294, 326, 484, 508, 596, 750, 1095, 1214, 1273, 1280, 1388, 1623, 1653, 1750, 1991, 2058, 2071, 2276, 2482, 2580, 2722, 2733, 2763, 2855, 2884, 2903, 2952, 3134, 3184, 3254, 3297, 3366, 3380, 3614, 3664, 3763, 3988, 4073, 4125, 4198, 4914, 4935, 5000, 5064, 5119, 5148, 5327, 5649, 5682, 5745, 5767, 5775, 5927, 5955, 6131, 6159, 6243, 6252, 6359, 6380, 6592, 6602, 6761, 6818, 6851, 6993, 7010, 7088, 7101, 7139, 7332, 7393, 7456, 7504, 7556, 7630, 7633, 7673, 7715, 7811, 8003, 8019, 8200, 8270, 8364, 8441, 8680, 8693, 8733, 8745, 8845, 8955, 9199, 9298, 9321, 9423, 9456, 9709, 9715, 9736, 9756, 9794, 9890, 9920, 10100, 10167, 10178, 10218, 10231, 10247, 10258, 10341, 10380, 10508, 10539, 10553, 10643, 10676, 10684, 10760, 10773, 10806, 10833, 10838, 10843, 10895, 11013, 11035, 11070, 11164, 11227, 11311, 11468, 11540, 11588, 11770, 11778, 11808, 11834, 11896, 11924, 11974, 12134, 12147, 12188, 12221, 12288, 12328, 12407, 12430, 12444, 12475, 12508, 12532, 12612, 12726, 12754, 1

In [36]:
print(listtrue.__len__())

260


In [37]:
print(listtrue)

[1250, 1384, 1590, 1657, 2107, 3319, 3369, 3519, 3604, 3917, 4222, 5423, 5445, 6768, 6923, 7199, 8067, 9185, 9426, 9498, 9534, 10924, 11324, 12514, 13147, 13150, 13210, 13280, 14051, 14611, 15203, 15301, 15409, 16387, 16436, 16478, 16952, 17027, 17203, 18396, 19149, 19192, 19220, 19421, 19477, 19499, 19735, 20062, 20571, 21044, 21350, 21513, 21680, 22247, 23328, 24100, 24505, 24952, 25158, 25604, 25763, 25830, 26374, 26573, 26770, 27234, 27612, 27614, 27736, 27986, 28097, 28474, 28869, 28916, 29071, 29360, 30669, 31004, 32577, 32631, 33371, 33668, 34163, 34311, 34437, 34518, 34819, 35015, 35560, 35606, 35878, 35916, 36127, 36255, 36349, 36548, 36977, 37121, 37221, 37826, 38541, 38798, 39431, 39496, 39620, 39810, 40281, 40336, 42225, 42443, 42461, 42509, 42998, 43116, 44440, 44886, 45769, 46256, 46411, 46478, 46949, 47271, 47289, 47786, 48147, 48163, 48271, 48346, 48456, 48517, 49018, 50230, 50491, 50506, 51580, 51925, 52416, 52440, 53147, 54434, 54714, 54972, 55215, 55578, 55709, 56176

In [38]:
differents_locations.loc[1250]["seller_address"]

{'comment': '',
 'longitude': -58.6167212,
 'id': 73143250,
 'country': {'name': 'Argentina', 'id': 'AR'},
 'address_line': 'san martin SN',
 'latitude': -34.6558611,
 'search_location': {'neighborhood': {'name': '', 'id': ''},
  'state': {'name': 'Bs.As. G.B.A. Oeste', 'id': 'TUxBUEdSQWVmNTVm'},
  'city': {'name': 'Morón', 'id': 'TUxBQ01PUmI1NTBj'}},
 'zip_code': '1708',
 'city': {'name': 'Morón', 'id': 'TUxBQ01PUmI1NTBj'},
 'state': {'name': 'Buenos Aires', 'id': 'AR-B'}}

In [39]:
differents_locations.loc[1250]["geolocation"]

{'latitude': -34.6477684, 'longitude': -58.6138669}

In [44]:
def json_get_country(row):
    country = row["seller_address"]["country"]["name"]
    if country != None and country != "":
        return country
    return None


In [45]:
df["country"] = df.apply(json_get_country, axis=1)



In [47]:
df["country"].value_counts()

country
Argentina    99997
Name: count, dtype: int64

In [46]:
df.head()

,seller_address,warranty,sub_status,condition,seller_contact,deal_ids,base_price,shipping,non_mercado_pago_payment_methods,seller_id,...,catalog_product_id,subtitle,initial_quantity,start_time,permalink,geolocation,sold_quantity,available_quantity,same_info_0_45,country
0,"{'comment': '', 'longitude': -58.3986709, 'id'...",None,[],new,None,[],80.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",74952096,...,NaN,NaN,1,2015-09-05 20:42:53+00:00,http://articulo.mercadolibre.com.ar/MLA-578052...,"{'latitude': -34.6280698, 'longitude': -58.398...",0,1,True,Argentina
1,"{'comment': '', 'longitude': -58.5059173, 'id'...",NUESTRA REPUTACION,[],used,None,[],2650.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",42093335,...,NaN,NaN,1,2015-09-26 18:08:30+00:00,http://articulo.mercadolibre.com.ar/MLA-581565...,"{'latitude': -34.5935524, 'longitude': -58.505...",0,1,True,Argentina
2,"{'comment': '', 'longitude': -58.4143948, 'id'...",None,[],used,None,[],60.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",133384258,...,NaN,NaN,1,2015-09-09 23:57:07+00:00,http://articulo.mercadolibre.com.ar/MLA-578780...,"{'latitude': -34.6233907, 'longitude': -58.414...",0,1,True,Argentina
3,"{'comment': '', 'longitude': -58.4929208, 'id'...",None,[],new,None,[],580.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",143001605,...,NaN,NaN,1,2015-09-28 18:47:56+00:00,http://articulo.mercadolibre.com.ar/MLA-581877...,"{'latitude': -34.6281894, 'longitude': -58.492...",0,1,True,Argentina
4,"{'comment': '', 'longitude': -58.5495042, 'id'...",MI REPUTACION.,[],used,None,[],30.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",96873449,...,NaN,NaN,1,2015-08-24 22:07:20+00:00,http://articulo.mercadolibre.com.ar/MLA-576112...,"{'latitude': -34.6346547, 'longitude': -58.549...",0,1,True,Argentina


In [50]:
df["condition"].isna().value_counts()

condition
False    100000
Name: count, dtype: int64